In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import findspark
findspark.init()

In [8]:
sparkSession = SparkSession.builder.appName("csv4").getOrCreate()

In [9]:
data = sparkSession.read.load("hdfs://localhost:9000/db-cw/trains.csv", format="csv",sep=",", inferSchema="true", header="true")
data.registerTempTable("train")
data = sparkSession.read.load("hdfs://localhost:9000/db-cw/tickets.csv", format="csv",sep=",", inferSchema="true", header="true")
data.registerTempTable("ticket")
data = sparkSession.read.load("hdfs://localhost:9000/passengers.csv", format="csv",sep=",", inferSchema="true", header="true")
data.registerTempTable("passenger")

/home/timerlan/.local/lib/python3.10/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [11]:
res = sparkSession.sql("""
    select sum(t.price) as total_sales from ticket t
    where t.date_purchase >= to_date('2010-01-01', 'yyyy-mm-dd') AND t.date_purchase <= to_date('2015-01-01', 'yyyy-mm-dd')
""")
res.show(20, False)

+-----------+
|total_sales|
+-----------+
|806918.72  |
+-----------+



In [6]:
sparkSession.stop()

### Spark stat

== Physical Plan ==
AdaptiveSparkPlan (13)
+- == Final Plan ==
   * HashAggregate (7)
   +- ShuffleQueryStage (6), Statistics(sizeInBytes=128.0 B, rowCount=8)
      +- Exchange (5)
         +- * HashAggregate (4)
            +- * Project (3)
               +- * Filter (2)
                  +- Scan csv  (1)
+- == Initial Plan ==
   HashAggregate (12)
   +- Exchange (11)
      +- HashAggregate (10)
         +- Project (9)
            +- Filter (8)
               +- Scan csv  (1)


(1) Scan csv 
Output [2]: [price#158, date_purchase#159]
Batched: false
Location: InMemoryFileIndex [hdfs://localhost:9000/db-cw/tickets.csv]
PushedFilters: [IsNotNull(date_purchase), GreaterThanOrEqual(date_purchase,2010-01-01 00:00:00.0), LessThanOrEqual(date_purchase,2015-01-01 00:00:00.0)]
ReadSchema: struct<price:double,date_purchase:timestamp>

(2) Filter [codegen id : 1]
Input [2]: [price#158, date_purchase#159]
Condition : ((isnotnull(date_purchase#159) AND (date_purchase#159 >= 2010-01-01 00:00:00)) AND (date_purchase#159 <= 2015-01-01 00:00:00))

(3) Project [codegen id : 1]
Output [1]: [price#158]
Input [2]: [price#158, date_purchase#159]

(4) HashAggregate [codegen id : 1]
Input [1]: [price#158]
Keys: []
Functions [1]: [partial_sum(price#158)]
Aggregate Attributes [1]: [sum#206]
Results [1]: [sum#207]

(5) Exchange
Input [1]: [sum#207]
Arguments: SinglePartition, ENSURE_REQUIREMENTS, [plan_id=300]

(6) ShuffleQueryStage
Output [1]: [sum#207]
Arguments: 0

(7) HashAggregate [codegen id : 2]
Input [1]: [sum#207]
Keys: []
Functions [1]: [sum(price#158)]
Aggregate Attributes [1]: [sum(price#158)#201]
Results [1]: [toprettystring(sum(price#158)#201, Some(Europe/Moscow)) AS toprettystring(total_sales)#204]

(8) Filter
Input [2]: [price#158, date_purchase#159]
Condition : ((isnotnull(date_purchase#159) AND (date_purchase#159 >= 2010-01-01 00:00:00)) AND (date_purchase#159 <= 2015-01-01 00:00:00))

(9) Project
Output [1]: [price#158]
Input [2]: [price#158, date_purchase#159]

(10) HashAggregate
Input [1]: [price#158]
Keys: []
Functions [1]: [partial_sum(price#158)]
Aggregate Attributes [1]: [sum#206]
Results [1]: [sum#207]

(11) Exchange
Input [1]: [sum#207]
Arguments: SinglePartition, ENSURE_REQUIREMENTS, [plan_id=285]

(12) HashAggregate
Input [1]: [sum#207]
Keys: []
Functions [1]: [sum(price#158)]
Aggregate Attributes [1]: [sum(price#158)#201]
Results [1]: [toprettystring(sum(price#158)#201, Some(Europe/Moscow)) AS toprettystring(total_sales)#204]

(13) AdaptiveSparkPlan
Output [1]: [toprettystring(total_sales)#204]
Arguments: isFinalPlan=true